# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our loss function.
After training, we will need these parameters
in order to make future predictions.
Additionally, we will sometimes wish
to extract the parameters
perhaps to reuse them in some other context,
to save our model to disk so that
it may be executed in other software,
or for examination in the hope of
gaining scientific understanding.

Most of the time, we will be able
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on deep learning frameworks
to do the heavy lifting.
However, when we move away from
stacked architectures with standard layers,
we will sometimes need to get into the weeds
of declaring and manipulating parameters.
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Sharing parameters across different model components.


In [45]:
import torch
from torch import nn
import torchvision
from torchvision import transforms

(**We start by focusing on an MLP with one hidden layer.**)


In [8]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## [**Parameter Access**]
:label:`subsec_param-access`

Let's start with how to access parameters
from the models that you already know.


When a model is defined via the `Sequential` class,
we can first access any layer by indexing
into the model as though it were a list.
Each layer's parameters are conveniently
located in its attribute.


We can inspect the parameters of the second fully connected layer as follows.


In [9]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.1319, -0.2411,  0.1723,  0.0130,  0.1349, -0.3048, -0.2249,  0.3502]])),
             ('bias', tensor([-0.1452]))])

We can see that this fully connected layer
contains two parameters,
corresponding to that layer's
weights and biases, respectively.


### [**Targeted Parameters**]

Note that each parameter is represented
as an instance of the parameter class.
To do anything useful with the parameters,
we first need to access the underlying numerical values.
There are several ways to do this.
Some are simpler while others are more general.
The following code extracts the bias
from the second neural network layer, which returns a parameter class instance, and
further accesses that parameter's value.


In [10]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.1452]))

Parameters are complex objects,
containing values, gradients,
and additional information.
That is why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.


In [11]:
net[2].weight.grad == None

True

### [**All Parameters at Once**]

When we need to perform operations on all parameters,
accessing them one-by-one can grow tedious.
The situation can grow especially unwieldy
when we work with more complex, e.g., nested, modules,
since we would need to recurse
through the entire tree to extract
each sub-module's parameters. Below we demonstrate accessing the parameters of all layers.


In [12]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## [**Tied Parameters**]

Often, we want to share parameters across multiple layers.
Let's see how to do this elegantly.
In the following we allocate a fully connected layer
and then use its parameters specifically
to set those of another layer.
Here we need to run the forward propagation
`net(X)` before accessing the parameters.


In [13]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


This example shows that the parameters
of the second and third layer are tied.
They are not just equal, they are
represented by the same exact tensor.
Thus, if we change one of the parameters,
the other one changes, too.


You might wonder,
when parameters are tied
what happens to the gradients?
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
during backpropagation.


## Summary

We have several ways of accessing and tying model parameters.


## Exercises

1. Use the `NestMLP` model defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Construct an MLP containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?


In [30]:
X = torch.rand((2,4))
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Sequential(nn.LazyLinear(64), nn.ReLU(), nn.LazyLinear(32), nn.ReLU())
        self.out = nn.LazyLinear(16)

    def forward(self, X):
        return self.out(self.hidden(X))\
        
net = NestMLP()
net(X)
[(name, param.shape) for name, param in net.named_parameters()]

[('hidden.0.weight', torch.Size([64, 4])),
 ('hidden.0.bias', torch.Size([64])),
 ('hidden.2.weight', torch.Size([32, 64])),
 ('hidden.2.bias', torch.Size([32])),
 ('out.weight', torch.Size([16, 32])),
 ('out.bias', torch.Size([16]))]

In [88]:
shared = nn.Linear(16, 16)
shared_net = nn.Sequential(nn.Flatten(), nn.LazyLinear(16), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(16,10))
trans = transforms.Compose([transforms.Resize((28,28)), transforms.ToTensor()])
train = torchvision.datasets.FashionMNIST(root = '../data', transform = trans)
val = torchvision.datasets.FashionMNIST(root = '../data', train=False, transform = trans)
training_loader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
validation_loader = torch.utils.data.DataLoader(val, batch_size=4, shuffle=False)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(shared_net.parameters(), lr = 0.001, momentum = 0.9)

def train_one_epoch(): 
    loss = 0
    loss_per_batch = 0.
    for i, data in enumerate(training_loader):
        optimizer.zero_grad()
        inputs, labels  = data
        y = shared_net(inputs)
        loss = loss_fn(y, labels)
        loss.backward()
        optimizer.step()
        loss_per_batch += loss
        if i % 1000 == 999:
            print("batch:", i, "loss_per_batch:", loss_per_batch/1000)
            loss_per_batch = 0
    return loss

In [89]:
epochs = 5
for i in range(epochs):
    print("EPOCH", i)
    shared_net.train(True)
    cur_loss = train_one_epoch()

    shared_net.eval()
    running_vloss = 0.
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = shared_net(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print("LOSS train:", cur_loss, "val:", avg_vloss)


EPOCH 0
batch: 999 loss_per_batch: tensor(1.9159, grad_fn=<DivBackward0>)
batch: 1999 loss_per_batch: tensor(1.0555, grad_fn=<DivBackward0>)
batch: 2999 loss_per_batch: tensor(0.7976, grad_fn=<DivBackward0>)
batch: 3999 loss_per_batch: tensor(0.7371, grad_fn=<DivBackward0>)
batch: 4999 loss_per_batch: tensor(0.6425, grad_fn=<DivBackward0>)
batch: 5999 loss_per_batch: tensor(0.6510, grad_fn=<DivBackward0>)
batch: 6999 loss_per_batch: tensor(0.6180, grad_fn=<DivBackward0>)
batch: 7999 loss_per_batch: tensor(0.5586, grad_fn=<DivBackward0>)
batch: 8999 loss_per_batch: tensor(0.5954, grad_fn=<DivBackward0>)
batch: 9999 loss_per_batch: tensor(0.5551, grad_fn=<DivBackward0>)
batch: 10999 loss_per_batch: tensor(0.5508, grad_fn=<DivBackward0>)
batch: 11999 loss_per_batch: tensor(0.4888, grad_fn=<DivBackward0>)
batch: 12999 loss_per_batch: tensor(0.4988, grad_fn=<DivBackward0>)
batch: 13999 loss_per_batch: tensor(0.5232, grad_fn=<DivBackward0>)
batch: 14999 loss_per_batch: tensor(0.4943, grad_fn

[Discussions](https://discuss.d2l.ai/t/57)
